### What Are We Doing?

* Architecture 
* Provisioning?
* Provisioning EC2 web-server with PostGreSQL
    * Default User/Table will be synced on deploy
* Provisioning EC2 web-server with Flask API
    * Application server directly talks to PostGreSQL with SQLAlchemy
* Provisioning EC2 web-server with Flask Python application 
    * Application server talks with REST API
    * Responsible for rendering the templates and responding to jquery calls
* Just Automate it
* Hit the public domain on static WebServer

### Architecture

![alt text](arch.png "Architecture")

Key things to notice

1. Keep a modular architecture for horizontal scalability and rapid .

When we 

2. Let the good things do what it's good at 

There are 2 roles here, the web-server and the application-server. 
The role of the web-server is to send web-requests to the appropriate spot, to handle request/response threads, and to redirect when necessary. The role of the application-server is to receive a request from the web-server, do somethinge with it, and then send a response back to the web-server. You can definitely set up an application-server to do what a web-server does but then you will need to write code in your application-server to handle 1000X requests-per-minute and to route properly and to properly set up all the virtual-hosts and serve up static files. You can also have a web-server act as an application-server if you only allow it to be serve static files

============================

The are two primary open-source web-servers in the wild: Nginx and Apache.

We will be using Nginx. 
Why? 
It's faster, it's easier to set up, it's production tested.


### Provisioning

What is provisioning again? 

Wikipedia: Server provisioning is a set of actions to prepare a server with appropriate systems, data and software, and make it ready for network operation. 

### Provisioning the PostGreSQL Database - Check

In the last lab, we built out a PostGreSQL Database.   
Let's make sure we have the user `ubuntu`, the password `cs207password`, and the database `ubuntu`.


### Provisioning the Flask API Server

In the lecture on APIs, we built a Flask API that has 1 basic model. We will keep this model, rename it, and then repackage it and deploy it somewhere.

How do we test this? 
Easy!
Check it out: http://flask.pocoo.org/docs/0.11/testing/

In [ ]:
import logging
from flask import Flask, request, abort, jsonify, make_response
from flask.ext.sqlalchemy import SQLAlchemy, DeclarativeMeta
from json import JSONEncoder

log = logging.getLogger(__name__)


class ProductJSONEncoder(JSONEncoder):

    def default(self, obj):
        if isinstance(obj.__class__, DeclarativeMeta):
            return obj.to_dict()
        return super(ProductJSONEncoder, self).default(obj)


app = Flask(__name__)
app.json_encoder = ProductJSONEncoder

# This can all go into a config file but is here for reference.
user = 'ubuntu'
password = 'cs207password'
host = 'localhost'
port = '5432'
db = 'ubuntu' 
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(user, password, host, port, db)
app.config['SQLALCHEMY_DATABASE_URI'] = url # 'sqlite:////tmp/tasks.db'
db = SQLAlchemy(app)


class Task(db.Model):
    __tablename__ = 'tasks'

    task_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    action = db.Column(db.String(80), nullable=False)

    def __repr__(self):
        return '<User %r>' % self.task_id

    def to_dict(self):
        return dict(action=self.action, task_id=self.task_id)


@app.route('/tasks', methods=['GET'])
def get_all_tasks():
    log.info('Getting all Tasks')
    return jsonify(dict(tasks=Task.query.all()))


@app.route('/tasks/<int:task_id>', methods=['GET'])
def get_task_by_id(task_id):
    task = Task.query.filter_by(task_id=task_id).first()
    if task is None:
        log.info('Failed to get Task with task_id=%s', task_id)
        abort(404)
    log.info('Getting Task with task_id=%s', task_id)
    return jsonify({'task': task})


@app.route('/tasks', methods=['POST'])
def create_task():
    if not request.json or 'action' not in request.json:
        abort(400)
    log.info('Creating Task with action=%s', request.json['action'])
    prod = Task(action=request.json['action'])
    db.session.add(prod)
    db.session.commit()
    return jsonify({'op': 'OK', 'task': prod}), 201


@app.route('/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    if not request.json or 'action' not in request.json:
        log.info('Could not update. Invalid params')
        abort(400)

    task = Task.query.filter_by(task_id=task_id).first()
    if task is None:
        log.info('Could not find Task id=%s to update', task_id)
        abort(404)

    action = request.json['action']
    log.info('Updating Task id=%s with action %s', task_id, action)
    task.action = action
    db.session.commit()
    return jsonify({'op': 'OK', 'task': task}), 201


@app.route('/tasks/<int:task_id>', methods=['DELETE'])
def remove_task(task_id):
    task = Task.query.filter_by(task_id=task_id).first()
    if task is None:
        abort(404)

    log.info('Deleting Task with id=%s', task_id)
    db.session.delete(task)
    db.session.commit()
    return jsonify({'op': 'OK'})


@app.route('/tasks', methods=['DELETE'])
def remove_all_tasks():
    tasks = Task.query.all()
    for task in tasks:
        db.session.delete(task)

    log.info('Deleted all Tasks!')
    db.session.commit()
    return jsonify({'op': 'OK'})


@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)


if __name__ == '__main__':
    logging.basicConfig(level=logging.DEBUG)
    db.create_all()
    app.run(port=5001)

### Provisioning the Web Application Server

The server is a simple MVC with a REST backed M, client-side V and flask C.
If we have static assets, we can let NginX do the heavy lifting.

Here's the webservice we will use!

In [ ]:

<body>
  <div id="myDIV" class="header">
    <h2>Simple To Do List</h2>
    <input type="text" id="taskInput" placeholder="Title...">
    <span onclick="createNewTask()" class="addBtn">New Task</span>
  </div>

  <ul id="myUL">
    {% for task in tasks %}
      <li id="{{task.task_id}}"> 
        <span class="action"> {{task.action}} </span>
        <span class="updater"> ↺  </span>
        <span class="close"> ✘ </span>
      </li>
    {% endfor %}
  </ul>
</body>

<script>
/*
 * Deletes a <li> task when clicking on the "X" button
 */
function deleteTask (liTag) {
    var taskId = liTag.attr('id');
    $.ajax({
      url: '/taskop',
      method: 'POST',
      contentType: 'application/json; charset=utf-8',
      dataType: 'json',
      data: JSON.stringify({ op: "DELETE", 
                             action: "Some Action",
                             task_id: parseInt(taskId)})
    })
    .done(function (result) {
      liTag.remove();
    });
}
/*
 * Create a new <li> task when clicking on the "Add" button
 */
function createNewTask() {
  var inputValue = $("#taskInput").val();
  if (inputValue === '') {
      alert("You must write something!");
  } else {
    console.log('Creating new task', inputValue);
    $.ajax({
      url: '/taskop',
      method: 'POST',
      contentType: 'application/json; charset=utf-8',
      dataType: 'json',
      data: JSON.stringify({ op: "ADD", action: inputValue })
    })
    .done(function (result) {
      // XXX: Explain chaining
      var newTask = result.task;
      $('<li/>', {
        id: newTask.task_id,
        text: newTask.action,
      }).appendTo('#myUL')
      // Add the span
      $('<span/>', {
        text: "✘ ",
        "class": "close"
      })
      .appendTo('#' + newTask.task_id)
      .click(function() {
          var liTag = $(this).parent('li');
          deleteTask(liTag);
      });
      // Remove the input that you just entered
      $("#taskInput").val('');
    });
  }
}
$(document).ready(function () {
  // Every close button when clicked will delete itself from the list
  // and submit a call to the API to self-delete
  $('.close').each(function (i, el) {
    el.onclick = function() {
        var liTag = $(this).parent('li');
        deleteTask(liTag);
    };
  })
  // Add a "checked" symbol when clicking on a list item
  //XXX: Make this jquery
  var list = document.querySelector('ul');
  list.addEventListener('click', function(ev) {
    if (ev.target.tagName === 'LI') {
      ev.target.classList.toggle('checked');
    }
  }, false);
});
</script>
</html>

Finally, after going through the code, simple ssh into your ec2 server as the superuser and run this
```
git clone https://github.com/fyeung86/eztodo.git
bash bin/server-setup.sh
```

In [ ]:
# Voila! You have a client, a REST API